# はじめての Docker for SageMaker

#### ノートブックに含まれる内容

- Docker の概要について説明
- Docker の基本的な使いかた

#### ノートブックで使われている手法の詳細

- Docker

## 概要

機械学習モデルの学習・推論を行う際には，各環境で使用するフレームワークやライブラリのバージョン，依存関係を完全に揃える必要があります．そうでないと正しく動かなかったり，動くけれども想定しない挙動を示すことがあるためです．ですが，機械学習の環境は非常に入り組んでおり，またバージョンアップも頻繁に行われるため，これを正しく管理することは非常に困難でした．

[Docker](https://www.docker.com/) を使うことにより，こうした問題をスマートに解決することができます．Dockerは完全な仮想化を行うものではなく，[Linux containers](https://en.wikipedia.org/wiki/LXC)を用いて、アプリケーションのパッケージ化や実行を容易にします．比較的軽量なため，アプリケーションコンテナが高速に立ち上がります．

Dockerfile に必要な設定を記述することで，また DockerHub や Amazon ECS のようなリポジトリ管理の仕組みを利用することで，これらの構成管理が驚くほど便利になります．カーネル 3.10 以降の Linux 上であれば，OS に依存することなく，どこでも同じようにソフトウェアを動かすことができます．また NVIDIA 社が提供する [`nvidia-docker`](https://github.com/NVIDIA/nvidia-docker) という Docker プラグインを利用することで，より簡単にコンテナからの GPU 利用を行うことができるようになります．

このノートブックでは，[Docker](https://www.docker.com/) の基本的な使い方について概観します．SageMaker でもスケーラブルな学習ジョブ，および推論エンドポイントの基盤として Docker を用いています．ノートブックを一通り終えることで，Docker の基本を理解し，SageMaker の利用に役立てつことができるようになります．

## Docker の基本

Docker コンテナを立ち上げるためには，イメージと呼ばれる，ベースとなるバイナリファイルが必要です．Dockerfile と呼ばれる設定ファイルをビルドすることで，Docker イメージが生成されます．いったん Docker イメージができたら，そこから `docker run` コマンドを実行することで，当該イメージをベースにコンテナを作成し，動作させることができます．

### コンテナのライフサイクル

Docker コンテナのライフサイクルは，以下のような図で表すことができます．イメージを指定して，`run` / `create` で実体としてのコンテナを生成し，あとは `start` / `stop` で動作を行い，最終的には `rm` で破棄します．同一のイメージから，コンテナを複数生成させることが可能です．またコンテナを破棄しても，元となるイメージが削除されるわけではありません．

![Container Lifecycle](container_lifecycle.png)

### イメージのライフサイクル

また，Docker イメージのライフサイクルも同じように以下で表すことができます．新しいイメージを作る際には，設定が書かれた dockerfile からイメージをビルドするだけでなく，コンテナに変更を加えた結果を新しいイメージとしてコミットすることもできます．作成したイメージは，ローカルストレージに保存するだけでなく，Amazon ECR のようなレジストリサービスに登録することができます．

![Container Lifecycle](image_lifecycle.png)



## セットアップ

SageMaker で提供されるノートブックインスタンスには，Docker および nvidia-docker プラグインがあらかじめインストールされています．以下のコマンドを実行して，バージョンを確認してみます．

In [ ]:
!docker --version

In [ ]:
!nvidia-docker --version

次に，`--help` で Docker のコマンド一覧を確認しましょう．

In [ ]:
!docker --help

Docker コマンドは，以下のようなフォーマットになります．

`docker [OPTIONS] COMMAND [arg...]` 

上記には非常にたくさんのコマンドがありますが，主要なものはそれほど多くないので，心配する必要はありません．

## コンテナのライフサイクルを理解する

それでは，これから実際に Docker を使っていきましょう．
まず最初に，`images` コマンドで，手元にあるイメージの一覧を確認します．
まだ何もしていないので，手元にイメージが保存されていないことが確認できるかと思います．

In [ ]:
!docker images

それでは，非常にシンプルなイメージを取得しましょう．以下のコマンドを打つと，DockerHub という Docker イメージを共有するパブリックリポジトリから，[hello-world](https://hub.docker.com/_/hello-world/) というイメージを取得します．

In [ ]:
!docker pull hello-world

あらためて `docker images` を実行すると，今度はイメージが手元にあることが確認できます．

In [ ]:
!docker images

それでは次に，`ps` コマンドで，ローカルに存在するコンテナの一覧を表示してみましょう．

In [ ]:
!docker ps -a

まだイメージを落としてきただけで，コンテナは作成していないので，一覧には何も出てきません．それでは，次に `create` コマンドを打って，hello-world イメージからコンテナを作成しましょう．

In [ ]:
!docker create hello-world

生成されたコンテナの，SHA256 UUID が表示されます．それではもう一度 `ps` コマンドを実行して，改めてコンテナ一覧を確認しましょう．

In [ ]:
!docker ps -a

STATUS の欄が Created になっているように，この時点では，コンテナはまだ動作していません（コンテナの STATUS については，このノートブックの冒頭にある，コンテナのライフサイクルの図を参照してください）．それでは，続いてこのコンテナを実際に動かしてみましょう．`start` コマンドを打って，コンテナを実際に動かします．コマンドは以下の形になりますが，ここでコンテナ ID が必要になります．

`docker start CONTAINER_ID`

さきほど作成したコンテナの ID は，上で実行した `docker ps -a` の出力から取得することができます．出力の最初のカラムにある，12 桁の英数字がそれです．以下のコマンドについて，**コンテナ ID を実際の値に置き換えて**実行してみてください．

In [ ]:
# CONTAINER IDを、上で実行したdocker ps -aコマンドの結果からコピー
!docker start 6ffe329ab76d

**`start`**コマンドを使用した結果，SHA256 UUID が返ってきました．それでは，Docker コンテナの一覧を改めて確認しましょう．

In [ ]:
!docker ps -a

STATUS が変化して，Exited (0) となっているのがわかるかと思います．これはコンテナが実行を終えて，STOP の状態になっているということです．また COMMAND が "/hello" となっていますが，これはコンテナが実行されたときに，実際には中で `/hello` が実行されたことを意味します．このコンテナは，単に hello を表示するコマンドを実行するだけのコンテナですので，これは想定どおりの動きといえます．コマンドを実行し終えたら，自動でコンテナは終了します．

では次に，`run` コマンドを使ってコンテナを立ち上げてみましょう．今度は，イメージから直接コンテナの作成・起動まで行うので，引数にイメージ ID を指定します．そのため，まずは `docker images` でイメージ ID を確認します．

In [ ]:
!docker images

以下のコマンドについて，引数を得られた イメージ ID に置き換えて，`run` コマンドを実行します．

In [ ]:
!docker run f2a91732366c

実行が終わったら，再度コンテナの状態を確認しましょう．新しいコンテナと，先ほど hello-world から作成したコンテナの 2 つが確認できます．

In [ ]:
!docker ps -a

それでは，今しがた `run` コマンドで作成済したコンテナ ID を指定して，再度 `start` コマンドでコンテナを走らせてみましょう

In [ ]:
!docker start e7e013c50fa0

今度は，出力が表示されません．これは `start` コマンドと `run` コマンドの挙動の違いで，`run` は実行したコマンドの標準出力を，そのまま表示してくれます．これに対して `start` はデフォルトでは標準出力を行いません．標準出力させるためには，`--attach` オプションを付与する必要があります．

In [ ]:
!docker start --attach e7e013c50fa0

今度はちゃんと出力が表示されました．なお，`--attach` をつけないときの標準出力は，コンテナのログファイルに送られます．各コンテナはログファイルを保持しており，`logs` コマンドでこれにアクセスすることが可能です．

In [ ]:
!docker logs e7e013c50fa0 

`run` コマンドは便利ですが，実行するたびに新しいコンテナを作成するため，ずっと開発をしていると，大量のコンテナが作られてしまいがちです．コンテナが大量にあること自体には特に問題はありませんが，非常に煩雑で見通しが悪くなってしまいます．そこで，`ps` コマンドでコンテナ ID の一覧をリストアップして，`rm` コマンドですべて削除してみましょう．

In [ ]:
# コンテナ ID の一覧を作成
!docker ps -a | awk '{print $1}' | tail -n +2

In [ ]:
# 一覧にまとめたコンテナを rm コマンドで削除
!for cid in $(docker ps -a | awk '{print $1}' | tail -n +2);do docker rm $cid; done

改めて `ps` コマンドを実行して，コンテナが残っていないことを確認してください．

In [ ]:
!docker ps -a

なお，`run` コマンドに `--rm` オプションをつけて実行することで，コンテナの実行が終わったら削除することができます．このオプションによって，大量のコンテナで溢れる事態を避けることができます．

In [ ]:
!docker run --rm f2a91732366c

In [ ]:
!docker ps -a

## イメージのライフサイクルを理解する

ここまで，コンテナがどのように作られ，実行され，そして破棄されるかという一連の流れを理解できたかと思います．それでは引き続いて，コンテナの元になるイメージのライフサイクルについてみていきましょう．

Docker のイメージは，ベースとなるイメージをもとに，それに修正を加えることでつくっていきます．このとき，新しいイメージは元のイメージを全部コピーするのでなく，変更差分のみを保持します．ですので，どんどん変更を加えて新しいイメージをいくつも作っても，各イメージは変更差分のみのデータを保つため，ディスクスペースを無駄に消費しません．また，変更を加えて作った Docker イメージを，リポジトリで管理することができます．こちらについてはあとで述べます．

### コンテナの変更からの新しいイメージの作成

まずは，簡単な例として Docker から公式で提供されている httpd イメージを落として，これに修正を加えて新しいイメージを作成しましょう．

In [ ]:
!docker images

In [ ]:
!docker pull httpd

In [ ]:
!docker images

httpd イメージが手元にあることを確認できるかと思います．それでは，このイメージに対して `apt-get update` コマンドを実行させましょう．`run` コマンドで，イメージ ID の後に書いたコマンドが，コンテナを立ち上げた際に実行されます．

In [ ]:
!docker run 7239615c0645 apt-get update

In [ ]:
!docker ps -a

この状態では，まだ変更を加えたコンテナを作成，実行しただけで，イメージの形にはなっていません．作成したコンテナに対して `commit` コマンドを実行することで，変更をイメージとして保存することができます．ですがまず，`diff` コマンドを使って，コンテナに対して行われた変更点を確認しましょう．コンテナ ID は，上で表示されたコンテナ ID に，適宜置き換えてください．

In [ ]:
!docker diff f3423defbaab

ここで　`A`　は追加されたファイルまたはディレクトリを，`C` は作成されたものを，`D` は削除されたものを表します．それでは `commit` コマンドで，新しいイメージを作成します．`commit` コマンドでは，コンテナ ID の後に `IMAGE_NAME:TAG_NAME` を指定します．ここでは `IMAGE_NAME` に `myhttpd`，`TAG_NAME` に `updated` としましょう．

In [ ]:
!docker commit f3423defbaab myhttpd:updated

それでは早速，新しいイメージが作成できたことを確認しましょう．

In [ ]:
!docker images

### 共有用の tar ファイルの作成

Dockerには，イメージを他の人と共有するために用いる `tar` ファイルを作成するため方法が 2 つあります．

* `save` / `load` コマンドを使って*イメージ* `tar` ファイルを作成する方法．ファイル差分の情報も保存するが，データ量がやや大きくなる
* `export` / `import` コマンドを使って*コンテナ* `tar` ファイルを作成する方法．ファイル差分の情報は保存されないが，データ量がやや小さくなる

それではまず，`save` コマンドで，イメージの `tar` ファイルを保存してみましょう

In [ ]:
!docker save -o saved_image.tar myhttpd:updated

きちんと保存されたかを確認しましょう．

In [ ]:
!ls -lah |grep tar

イメージを保存できたので，既存の `myhttpd:updated` イメージを削除しましょう．

In [ ]:
!docker images

In [ ]:
!docker rmi ffe861dabc92

In [ ]:
!docker images

`myhttpd` イメージが消えたことが確認できます．それでは今度は，`load` コマンドで保存済みイメージを，再度読み込みましょう．

In [ ]:
!docker load -i saved_image.tar

In [ ]:
!docker images

`myhttpd:updated` というイメージが，再度確認できます．ここでは実行しませんが，`export` / `import` でも同様にコンテナに対して，保存と読み込みを行うことができます．

### Dockerfileによるイメージの作成

ここまで，コンテナに直接コマンドを実行することで変更を与え，それを保存してきました．ですが，これらの手順をすべてコードとして保持し，新しいイメージのビルドワークフローを自動化する，といった目的のために，`Dockerfile` を用います．これは，Dcoker イメージのビルド手順をコマンドの形で表現したテキストファイルになります．それでは，先ほど行なった変更を，`Dockerfile` で同様に行なってみましょう．

In [ ]:
%%bash
cat << EOF > Dockerfile
FROM httpd:latest
RUN apt-get update
ENTRYPOINT
EOF

`Dockerfile` の詳細なコマンドについては，[公式ドキュメント](https://docs.docker.com/engine/reference/builder/)をご覧ください．ここでは，上記ファイルで使用しているコマンドについてだけ解説します．

* `FROM`: ベースとなるイメージを指定
* `RUN`: ベースイメージに対して実行するコマンドを記述し，結果をコミット
* `ENTRYPOINT`: コンテナ実行時に引き渡されるコマンドを実行する際のベースとなるパスを指定

このファイルがあるフォルダ上で　`build` コマンドを実行することで，新しいイメージをビルドすることができます．`-t IMAGE_NAME:TAG_NAME` を引数として指定します．

In [ ]:
!docker build -t myhttpd:updated_dockerfile .

In [ ]:
!docker images

これで，`Dockerfile` をもとに新しいイメージが作成されたことを確認できました．では次に，`tag` コマンドを使って，既存のイメージに対して新しいタグを付与してみましょう．

In [ ]:
!docker tag myhttpd:updated_dockerfile myhttpd:renamed

In [ ]:
!docker images

典型的には、個人やローカル環境で利用する限り、イメージの名前はさほど大きな問題にはなりません。
しかし、ひとたびイメージを共有したり配布したりすると、Dockerが *従うべきイメージの命名規則* が存在しています。クリーンアップのため、**`rmi`**コマンドを実行しましょう。

### イメージリポジトリの利用

作成した Docker イメージは，他の人と共有することが可能です．デフォルトの `docker` リポジトリは ["DockerHub"](https://hub.docker.com/) と呼ばれる，コミュニティのパブリックリポジトリです．リポジトリは github と似たイメージで使用することができます．`pull` コマンドでイメージを取得し，また手元でビルドしたイメージを `push` でリポジトリに登録します．

Sagemaker で Docker イメージを利用する場合には，AWS が提供しているリポジトリサービスの Elastic Container Registry (ECR) を利用する形になります．ここでは，ECR 対象として，先ほど作ったイメージを登録してみましょう．まずは，ECR に登録するために，新しくタグをつけます．

その際に，リソース名が他の人とぶつからないようにします．ここでは，末尾に 2 桁の数字をつけることにします．**<span style="color: red;">`myhttpd-xx` の `xx` を指定された適切な数字に変更</span>**してから，以下のコマンドを実行してください．

In [ ]:
%%bash
account=$(aws sts get-caller-identity --query Account --output text)
region=$(aws configure get region)
repository="${account}.dkr.ecr.${region}.amazonaws.com/myhttpd-xx:renamed"
docker tag myhttpd:renamed ${repository}

In [ ]:
!docker images

次に，ECR のリポジトリを作成します．`aws cli` 経由で `aws ecr describe-repositories` コマンドを実行して，当該リポジトリの有無を確認します．存在しなければ，`aws ecr create-repository` コマンドで新しくリポジトリを作成します．

その際に，先ほどと同様 2 行目と 4 行目の **<span style="color: red;">`myhttpd-xx` の `xx` を指定された適切な数字に変更</span>**してください．

In [ ]:
%%bash
aws ecr describe-repositories --repository-names "myhttpd-xx" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "myhttpd-xx"
fi

続いて，docker クライアントから ECR リポジトリにイメージを `push` できるように，ECR に対して認証を行います．

In [ ]:
!$(aws ecr get-login --region us-east-1 --no-include-email)

`Login Succeeded` と表示されたら，早速 `push` コマンドでイメージを登録してみましょう．

In [ ]:
%%bash
account=$(aws sts get-caller-identity --query Account --output text)
region=$(aws configure get region)
repository="${account}.dkr.ecr.${region}.amazonaws.com/myhttpd-xx:renamed"
docker push ${repository}

そうしたら，実際にイメージが登録されたかを確認してみましょう．その際に，先ほどと同様 **<span style="color: red;">`myhttpd-xx` の `xx` を指定された適切な数字に変更</span>**してください．

In [ ]:
!aws ecr describe-images --repository-name myhttpd-xx

ここまで確認できたら，後片付けとして，リポジトリを削除しましょう．以下のコマンドを実行してください．その際に，先ほどと同様 **<span style="color: red;">`myhttpd-xx` の `xx` を指定された適切な数字に変更</span>**してください．

In [ ]:
!aws ecr delete-repository --repository-name myhttpd-xx --force

同様に，残っているコンテナとイメージもすべて削除してしまいましょう．

In [ ]:
!docker ps -a

In [ ]:
!for cid in $(docker ps -a | awk '{print $1}' | tail -n +2);do docker rm $cid; done

In [ ]:
!docker ps -a

In [ ]:
!docker images

In [ ]:
!for iid in $(docker images | awk 'BEGIN {OFS=":"} {print $1,$2}' | tail -n +2);do docker rmi $iid; done

In [ ]:
!docker images

以上で，イメージのライフサイクルまで含めて Docker の基本を理解できたかと思います．